# Base Line (character)

In [15]:
import os
from data_preparation import *
from model import *

## 1. load dataset

In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset, char2idx, idx2char, vocab = char_text2tf_data(filename='donald_tweets.txt',
                                                       seq_length=100,
                                                       batch_size=BATCH_SIZE,
                                                       buffer_size=BUFFER_SIZE)

## 2. build training model

In [17]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNIT = 1024
model = build_basic_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNIT,
    batch_size=BATCH_SIZE
)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           27392     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 107)           109675    
Total params: 4,075,371
Trainable params: 4,075,371
Non-trainable params: 0
_________________________________________________________________


In [18]:
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
OPTIMIZER = 'adam'
model.compile(optimizer=OPTIMIZER, loss=LOSS)

## 3. train

In [19]:
checkpoint_dir = './base_line_character_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/10
121/121 [==============================] - 4s 31ms/step - loss: 3.1190

'./base_line_character_checkpoints\\ckpt_10'

## 4. build predicting model

In [20]:
model = build_basic_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNIT, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [21]:
def generate_text(model, start_string, num_generate, split_string):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string + split_string.join(text_generated)


print(generate_text(model, u'@', 1000, ''))

@MTeDAS: @realDonaldTrump Surgent, SAY!
"@tavyTa: Trump Nicels is comyone with U.S. is loing a Slive with the shocling to @FoxNews. Enjoy!
My froe home endorsed increase sies to love with you all credibition: Just fall in U.S. history, est of you! #Trump2016
@realDonaldTrump Clinton Familian messaged by @drongeaturviewhe just out -- he campaign.
Obama driving Crooked is scarpan:
"@PatrickJ911985270: @realDonaldTrump" Thank you!
"@BkranHalk023: @realDonaldTrump getts for politicians. Not nice!
Thank you Pon. A @RavenMilorin DWS Thrim
RT @DRUDGO_RT @realDonaldTrump Parel, Paul Ryan. #UIPrimary
I am in honore.7% #debate #askyrump
Happy Barbara.
Topill review like a down" So lie!
Thank you Nairica Wakerouside: @HISATERomind Romney has no tried to have personer, she is rigged!
Thank you Kannot Stated of this &amp; well dont.
I dont blew a great guy, whan is lett borred lobbyists: By Trump"
"@ofcatce3rfin: @DindanAIHadar: #Trump2016 #MakeAmericaGreatAgain #Trump2016
Crooked Hi
